In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

In [129]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst
def get_last_line_as_numpy_array(file_path, dtype = float):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names
def getCylIdx(folder_path):
    namelist = list_files_with_prefix(folder_path, "Cyl_watercontent_")
    idxlist = np.full(len(namelist),-1,dtype = int)
    for nn, nl in enumerate(namelist):
        match = re.search(r'\d+', nl)
        number_str = match.group()
        number_int = int(number_str)
        idxlist[nn] = number_int
    return idxlist
# cylinder max konz (per cell)
def getData_(cid,gId,path2file):
    fileName = "Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

numPset = 99
nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 

def getData():
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    for pSet in range(numPset):
        print(pSet,end =", ")

        path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
        
        rr = getCylIdx(path2file)
        rr.sort()
        print('(',len(rr),')',end =", ")

        for cid in range(nKonz):
            Ginits = np.array([getData_( cid, gId,path2file) for gId in rr])          
            GiniAll[cid][pSet] = Ginits
        for cid in range(nKonz,volIdx):
            nodes_ =get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", dtype =float)
            GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
        #cid = nToGet -1
        Ginits = np.array([getData_( volIdx, gId,path2file) for gId in rr])
        GiniAll[volIdx][pSet] = Ginits
        lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        GiniAll[lenIdx][pSet] = lens[rr]
        sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float)
        assert len(sts) == len(lens)
        GiniAll[stIdx][pSet] = sts[rr]
        
        orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", dtype =int)
        globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", dtype =int)
        parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", dtype =int)
        ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", dtype =int)
        id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", dtype =int)
        id_roots = id_orgs[np.where(ot_orgs==2)]
        
        org2basenode = dict(zip(id_orgs, parentNidx))
        #org2ot = dict(zip(id_orgs, ot_orgs))
        isRoot = np.array([oo in id_roots for oo in orgidPerNode])
        
        orgidPerRoot = orgidPerNode[np.where(isRoot)]
        globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
        segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
        orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
        #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))
        
        relLens = np.zeros(len(segIdxRoot))
        #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        for myorg in orgIdxRoot:
            thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
            lenSegs = GiniAll[lenIdx][pSet]
            relLens[thensegs] = np.cumsum(lenSegs[thensegs])
            
            
        GiniAll[orgIdId][pSet] = orgIdxRoot
        GiniAll[relLenId][pSet] = relLens

    return GiniAll #cid pSet rr
    

In [130]:
GiniAll = getData()

0, ( 232 ), 1, ( 232 ), 2, ( 232 ), 3, ( 232 ), 4, ( 235 ), 5, ( 232 ), 6, ( 232 ), 7, ( 235 ), 8, ( 232 ), 9, ( 235 ), 10, ( 232 ), 11, ( 232 ), 12, ( 232 ), 13, ( 232 ), 14, ( 232 ), 15, ( 236 ), 16, ( 232 ), 17, ( 232 ), 18, ( 232 ), 19, ( 236 ), 20, ( 232 ), 21, ( 232 ), 22, ( 232 ), 23, ( 236 ), 24, ( 232 ), 25, ( 234 ), 26, ( 232 ), 27, ( 235 ), 28, ( 232 ), 29, ( 236 ), 30, ( 232 ), 31, ( 232 ), 32, ( 234 ), 33, ( 232 ), 34, ( 232 ), 35, ( 235 ), 36, ( 235 ), 37, ( 236 ), 38, ( 234 ), 39, ( 232 ), 40, ( 232 ), 41, ( 236 ), 42, ( 232 ), 43, ( 236 ), 44, ( 234 ), 45, ( 234 ), 46, ( 235 ), 47, ( 236 ), 48, ( 232 ), 49, ( 236 ), 50, ( 232 ), 51, ( 232 ), 52, ( 236 ), 53, ( 232 ), 54, ( 234 ), 55, ( 235 ), 56, ( 236 ), 57, ( 234 ), 58, ( 234 ), 59, ( 235 ), 60, ( 236 ), 61, ( 232 ), 62, ( 234 ), 63, ( 234 ), 64, ( 236 ), 65, ( 232 ), 66, ( 234 ), 67, ( 232 ), 68, ( 232 ), 69, ( 235 ), 70, ( 236 ), 71, ( 234 ), 72, ( 236 ), 73, ( 236 ), 74, ( 232 ), 75, ( 236 ), 76, ( 234 ), 77, ( 234

In [131]:
#assert np.array(GiniAll).shape == (nToGet, numPset, 232) # some have a couple of segments more

/tmp/ipykernel_1126/1273250247.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  assert np.array(GiniAll).shape == (nToGet, numPset, 232) #all same shape


AssertionError: 

In [132]:
np.array(GiniAll).shape

/tmp/ipykernel_1126/2873905747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(GiniAll).shape


(17, 99)

In [133]:
# get param set
paramsets =  pd.read_csv('../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
paramsets['k_C,S'] /=paramsets['theta']
paramsets['k_O,S'] /=paramsets['theta']
paramsets.drop(labels='theta', inplace=True, axis=1)
outputvalsname =  ['wat','cs','cl','coa','cod','cca','ccd','css2','co2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + paramsets.columns.tolist()+['pSet']
df = pd.DataFrame(columns=column_names)

for pset in range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_to_duplicate_df = pd.DataFrame(paramsets.iloc[pset]).T
    duplicated_rows_df = rows_to_duplicate_df.loc[np.repeat(rows_to_duplicate_df.index.values, N)]
    # Duplicate the specific rows N times
    #print('init dupl row shape',duplicated_rows_df.shape)
    for jj, outVn in enumerate(outputvalsname):
        duplicated_rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    duplicated_rows_df['pSet'] = pset
    # Concatenate the original DataFrame with the duplicated specific rows
    df = pd.concat([df, duplicated_rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

In [128]:
df

,wat,cs,cl,coa,cod,cca,ccd,css2,co2,yX,...,Y,Y_C,Y_O,CSS_max,k_sorp,CL_init,CS_init,pSet,co,cc
0,0.049072,2.639671e-07,0.000906,9.085835e-08,0.000009,3.364929e-07,8.915930e-06,0.000007,2.472739e-06,-0.298434,...,0.234902,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,0.000009,0.000009
1,0.008728,2.438039e-07,0.000852,9.081024e-08,0.000009,3.363607e-07,8.915956e-06,0.000007,2.469714e-06,0.043922,...,0.234902,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,0.000009,0.000009
2,0.010203,5.398630e-07,0.000925,9.154289e-08,0.000009,3.372123e-07,8.915583e-06,0.000007,2.483268e-06,-0.447197,...,0.234902,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,0.000009,0.000009
3,0.013479,2.466914e-07,0.000856,9.081228e-08,0.000009,3.363631e-07,8.915956e-06,0.000007,2.469792e-06,-0.104340,...,0.234902,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,0.000009,0.000009
4,0.010503,1.760157e-04,0.001104,8.208593e-06,0.000001,9.963765e-06,1.218063e-09,0.000061,2.295724e-05,-0.125120,...,0.234902,0.034053,0.469288,1.437543,0.039387,11.083551,0.006590,0,0.000010,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,0.256344,4.608546e-06,0.000990,8.697761e-07,0.000008,6.688528e-07,8.575915e-06,0.000012,5.246680e-07,0.601310,...,0.487350,0.173275,0.294169,1.421506,0.116672,11.870338,0.036058,1,0.000009,0.000009
460,0.018524,8.374195e-06,0.000990,1.087485e-06,0.000008,9.456851e-07,8.300378e-06,0.000012,5.334436e-07,-0.232992,...,0.487350,0.173275,0.294169,1.421506,0.116672,11.870338,0.036058,1,0.000009,0.000009
461,0.247872,3.999142e-06,0.000991,8.816617e-07,0.000008,6.063162e-07,8.638386e-06,0.000012,5.242397e-07,0.738255,...,0.487350,0.173275,0.294169,1.421506,0.116672,11.870338,0.036058,1,0.000009,0.000009
462,0.035059,4.471629e-06,0.000991,9.133746e-07,0.000008,6.188445e-07,8.625870e-06,0.000012,5.245975e-07,0.913513,...,0.487350,0.173275,0.294169,1.421506,0.116672,11.870338,0.036058,1,0.000009,0.000009


In [134]:
df.to_pickle('./cyl10to11_allroots2.pkl')